In [3]:
!pip install yfinance
!pip install fmp_python

  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23909 sha256=83f25e50bcdf65d1910c6b6239ac8fa290dce53a613299986ebe87c952fd4578
  Stored in directory: /home/jonghyeok/.cache/pip/wheels/ec/cc/c1/32da8ee853d742d5d7cbd11ee04421222eb354672020b57297
Successfully built yfinance


In [176]:
import yfinance as yf
from pandas_datareader import data as pdr
import os
from dotenv import load_dotenv
import typing
from fmp_python.fmp import FMP
import pandas as pd
import numpy as np
from urllib.request import urlopen
import json
import csv
import bs4 as bs
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


<h1>Setting FMP API Key</h1>

In [25]:
load_dotenv()
apikey = os.environ.get("apikey")

<h1>Getting SPY500 Tickers</h1>

In [26]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    ticker = ticker.replace("\n", "")
    ticker = ticker.replace(".", "-")
    tickers.append(ticker)

<h1>Getting FMP Data

In [132]:
#!/usr/bin/env python

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

In [133]:
def getData(datatype, ticker, apikey):
    url = ("https://financialmodelingprep.com/api/v3/"+datatype+"/"+ticker+"?&apikey="+apikey)
    jsonData = get_jsonparsed_data(url)
    return jsonData

In [131]:
spyKeyRatio

""


In [134]:
#Getting Ratio Data
spyKeyRatio = pd.DataFrame()
failTickers = []
for ticker in tickers:
    try:
        ratioJsondata = getData('ratios', ticker, apikey)
        sectorJsondata = getData('profile', ticker, apikey)
        df = pd.DataFrame.from_dict(ratioJsondata, orient='columns')
        df['industry']=sectorJsondata[0]['industry']
        df['sector']=sectorJsondata[0]['sector']
        df = df.drop(['period'], axis=1)
        spyKeyRatio = pd.concat([spyKeyRatio,df], axis=0)
    except:
        failTickers.append(ticker)
        pass

<h1>Getting Price Data from Yahoo</h1>

In [38]:
yf.pdr_override()
years = ["2016-01-29","2017-01-30","2018-01-30","2019-01-30","2020-01-30","2021-01-29"]
startDate = pd.to_datetime(years, format='%Y-%m-%d') 
yearlyPrice = pd.DataFrame()
for startDate in startDate:
    endDate = startDate + pd.DateOffset(days = 1)
    price = pdr.get_data_yahoo(tickers, start=startDate, end=endDate)
    price = price['Adj Close'].stack().reset_index()
    yearlyPrice = pd.concat([yearlyPrice,price], axis=0)

[*********************100%***********************]  505 of 505 completed

12 Failed downloads:
- OGN: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- OTIS: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- IR: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- LW: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- CTVA: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- DOW: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- FTV: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- UA: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- CARR: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- MRNA: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- FOX: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
- FOXA: Data doesn't exist for startDate = 1454054400, endDate = 1454140800
[**

Exception in thread Thread-3988:
Traceback (most recent call last):
  File "/home/jonghyeok/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/jonghyeok/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jonghyeok/.local/lib/python3.8/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/multi.py", line 169, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/multi.py", line 181, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/base.py", line 162, in history
    data = data.json()
  File "/home/jonghyeok/.local/lib/python3.8/site-packages/

[**********************89%******************     ]  450 of 505 completed

Exception in thread Thread-3997:
Traceback (most recent call last):
  File "/home/jonghyeok/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/jonghyeok/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jonghyeok/.local/lib/python3.8/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/multi.py", line 169, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/multi.py", line 181, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/base.py", line 162, in history
    data = data.json()
  File "/home/jonghyeok/.local/lib/python3.8/site-packages/

[**********************92%*******************    ]  464 of 505 completed

Exception in thread Thread-4013:
Traceback (most recent call last):
  File "/home/jonghyeok/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/jonghyeok/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jonghyeok/.local/lib/python3.8/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/multi.py", line 169, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/multi.py", line 181, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/base.py", line 162, in history
    data = data.json()
  File "/home/jonghyeok/.local/lib/python3.8/site-packages/

[**********************93%********************   ]  471 of 505 completed

Exception in thread Thread-4021:
Traceback (most recent call last):
  File "/home/jonghyeok/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/jonghyeok/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jonghyeok/.local/lib/python3.8/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/multi.py", line 169, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/multi.py", line 181, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "/home/jonghyeok/anaconda3/lib/python3.8/site-packages/yfinance/base.py", line 162, in history
    data = data.json()
  File "/home/jonghyeok/.local/lib/python3.8/site-packages/

[**********************99%***********************]  500 of 505 completed

KeyboardInterrupt: 

In [50]:
#priceData From Google Colab
yearlyPrice = pd.read_csv('priceData.csv')
yearlyPrice = yearlyPrice.drop("Unnamed: 0", axis=1)
yearlyPrice = yearlyPrice.rename(columns={"level_1": "ticker", "0":"price"})
yearlyPrice = yearlyPrice.sort_values(by=['ticker','Date']).reset_index(drop=True) 

In [60]:
years = ["2016-01-29","2017-01-30","2018-01-30","2019-01-30","2020-01-30","2021-01-29"]
startDate = pd.to_datetime(years, format='%Y-%m-%d') 
spyPrice = pd.DataFrame()
for startDate in startDate:
    endDate = startDate + pd.DateOffset(days = 1)
    price = pdr.get_data_yahoo("SPY", start=startDate, end=endDate)
    price = price['Adj Close']
    spyPrice = pd.concat([spyPrice,price], axis=0)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [107]:
yearlyPrice['Date'] = pd.to_datetime(yearlyPrice['Date'], format='%Y-%m-%d')
mergedPrice = yearlyPrice.merge(spyPrice,on='Date').sort_values(by=['ticker','Date']).reset_index(drop=True)
shifted = mergedPrice.shift(periods=1, axis="rows", fill_value=0)

<ipython-input-107-bd8f0fb27ab6>:3: FutureWarning: Passing <class 'int'> to shift is deprecated and will raise in a future version, pass Timestamp instead.
  shifted = mergedPrice.shift(periods=1, axis="rows", fill_value=0)


In [128]:
mergedPrice['eq'] = mergedPrice['ticker'].eq(shifted['ticker'])

In [125]:
subt = mergedPrice[['price','Adj Close']].subtract(shifted[['price','Adj Close']])
mergedPrice[['tickerC','spyC']]=subt[['price','Adj Close']] / mergedPrice[['price','Adj Close']]

In [126]:
priceComp = mergedPrice[mergedPrice['eq']==True].drop(['price','Adj Close','eq'], axis=1).reset_index(drop=True)
priceComp['diff'] = priceComp['ticekrC']-priceComp['spyC']

In [127]:
priceComp['date'] = priceComp['Date'] 

,Date,ticker,ticekrC,spyC,tickerC,diff
0,2017-01-30,A,0.218752,0.166568,0.218752,0.052184
1,2018-01-30,A,0.352731,0.207799,0.352731,0.144932
2,2019-01-30,A,0.041362,-0.033365,0.041362,0.074727
3,2020-01-30,A,0.113516,0.198586,0.113516,-0.085070
4,2021-01-29,A,0.303478,0.130778,0.303478,0.172700
...,...,...,...,...,...,...
2472,2017-01-30,ZTS,0.213172,0.166568,0.213172,0.046604
2473,2018-01-30,ZTS,0.312070,0.207799,0.312070,0.104271
2474,2019-01-30,ZTS,0.084968,-0.033365,0.084968,0.118333
2475,2020-01-30,ZTS,0.387120,0.198586,0.387120,0.188534


In [192]:
ratioData[ratioData['ticker']=='AAPL']

,ticker,date,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,...,priceFairValue,industry,sector,year,Date,ticekrC,spyC,tickerC,diff,outPerform
9,AAPL,2016-09-24,1.352669,1.049401,0.259271,26.665909,5.923304,32.589213,103.613369,-71.024156,...,5.018690,Consumer Electronics,Technology,2017,2017-01-30,0.218294,0.166568,0.218294,0.051726,False
10,AAPL,2017-09-30,1.276063,1.089670,0.201252,56.800671,12.563631,69.364302,126.927606,-57.563304,...,6.488908,Consumer Electronics,Technology,2018,2018-01-30,0.283075,0.207799,0.283075,0.075276,False
11,AAPL,2018-09-29,1.123843,0.986566,0.221733,67.332499,8.817631,76.150130,124.570214,-48.420084,...,9.815760,Consumer Electronics,Technology,2019,2019-01-30,0.004536,-0.033365,0.004536,0.037901,False
12,AAPL,2019-09-28,1.540126,1.384447,0.462022,64.258765,9.263639,73.522404,104.314077,-30.791673,...,12.709658,Consumer Electronics,Technology,2020,2020-01-30,0.497317,0.198586,0.497317,0.298731,True
13,AAPL,2020-09-26,1.363604,1.218195,0.360710,49.787534,8.741883,58.529418,91.048190,-32.518772,...,30.553901,Consumer Electronics,Technology,2021,2021-01-29,0.391677,0.130778,0.391677,0.260899,True


In [140]:
ratioData = pd.read_csv('ratio.csv')
ratioData = ratioData.drop('Unnamed: 0',axis=1).sort_values(by=['symbol','date'])

In [159]:
ratioData = ratioData.rename(columns={'symbol':'ticker'})

In [152]:
ratioData['year'] = pd.DatetimeIndex(ratioData['date']).year

In [153]:
ratioData['year'] = ratioData['year']+1
priceComp['year'] = pd.DatetimeIndex(priceComp['Date']).year

In [161]:
ratioData = ratioData.merge(priceComp, on=['ticker','year'])
ratioData['outPerform'] = ratioData['diff'] >= 0.1 

In [180]:
finalData = ratioData.drop(['year','Date','tickerC','spyC','ticekrC','diff','ticker','date','industry','sector'],axis=1)

In [181]:
y = finalData['outPerform'].values
X = finalData.drop('outPerform',axis=1).values

In [182]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [186]:
rf_stock_model = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=5)

In [187]:
rf_stock_model.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=5)

In [188]:
rf_stock_model.score(X_valid, y_valid)

0.765625